In [40]:
import pandas as pd

In [41]:
def report_string_to_dict(report):
    report_dict = {}
    metrics = report.split('\n')[0].split()
    for row in report.split('\n')[1:]:
        splitted = row.split()
        if len(splitted) > 4:
            class_name_size = len(splitted[0:-4])
            class_name = " ".join(splitted[0:-4])
            report_dict[class_name] = {}
            for metric_index in range(len(metrics)):
                metric = metrics[metric_index]
                metric_val = splitted[metric_index + class_name_size]
                report_dict[class_name][metric] = metric_val
        elif len(splitted) > 0:
            metric = splitted[0]
            report_dict[metric] = {}
            report_dict[metric][metrics[-2]] = splitted[1]
            report_dict[metric][metrics[-1]] = splitted[2]
    return report_dict

In [42]:
def csv_to_dataframe(csv_name, csv_folder = ""):
    if len(csv_folder) == 0:
        return pd.read_csv(csv_name + ".csv")
    else:
        return pd.read_csv(csv_folder + "/" + csv_name + ".csv")

In [43]:
def get_metrics_avg(csv_name, dataset_name, model_name):
    df = csv_to_dataframe(csv_name)
    df_for_dataset = df.loc[df['dataset'] == dataset_name]
    df_for_dataset_and_model = df.loc[df['model'] == model_name]
    reports_string = df_for_dataset_and_model['report'].tolist()
    reports = []
    for report in reports_string:
        reports.append(report_string_to_dict(report))

    num_of_instances = len(reports)

    accuracy_avg = 0
    f1_score_avg = 0
    f1_score_weighted_avg = 0

    for report in reports:
        accuracy_avg += float(report['accuracy']['f1-score'])
        f1_score_avg += float(report['macro avg']['f1-score'])
        f1_score_weighted_avg += float(report['weighted avg']['f1-score'])

    accuracy_avg = round(accuracy_avg / num_of_instances, 3)
    f1_score_avg = round(f1_score_avg / num_of_instances, 3)
    f1_score_weighted_avg = round(f1_score_weighted_avg / num_of_instances, 3)
    return accuracy_avg, f1_score_avg, f1_score_weighted_avg

In [44]:
get_metrics_avg('classic_models_results', 'caesarian.arff', 'perceptron')

(0.371, 0.239, 0.245)


In [ ]:
get_metrics_avg('classic_models_results', 'caesarian.arff', 'mlp')

In [ ]:
get_metrics_avg('classic_models_results', 'breast_tissue_data.csv', 'perceptron')

In [ ]:
get_metrics_avg('classic_models_results', 'breast_tissue_data.csv', 'mlp')